In [1]:
# import ase modules
import ase
import ase.io
import ase.build
import ase.visualize

# import numpy and other modules
import os
import numpy

In [4]:
Pb = ase.Atom(position=(0.5,0.5,0.5), symbol='Pb')
O1 = ase.Atom(position=(0.5, 0.5, 0.0), symbol='O')
O2 = ase.Atom(position=(0.5, 0.0, 0.5), symbol='O')
O3 = ase.Atom(position=(0.0, 0.5, 0.5), symbol='O')
Ti = ase.Atom(position=(0.0, 0.0, 0.0), symbol='Ti')
atoms = ase.Atoms((Pb, O1, O2, O3, Ti), cell=[1.0,1.0,1.0])
cell = atoms.get_cell()
cell *= 5.0                               # set the lattice constant
atoms.set_cell(cell, scale_atoms=True)    # sclae_atoms 
ase.visualize.view(atoms)
ase.io.write('PbTiO3_POSCAR', atoms, format='vasp')

In [2]:
PbTiO3 = ase.io.read('PbTiO3_POSCAR')
SrTiO3 = ase.io.read('SrTiO3_POSCAR')

# get the unitcell length of PbTiO3 in the z-direction
cell = PbTiO3.get_cell()
z_PbTiO3 = cell[2][2]

# get two-reps of PbTiO3 in the z-direction
PbTiO3 *= (1,1,2)

# get 3-reps of SrTiO3 in the z-direction
cell = SrTiO3.get_cell()
z_SrTiO3 = cell[2][2]
SrTiO3 *= (1,1,3)

# translate SrTiO3 in the z-direction by two times z_PbTiO3
SrTiO3.translate([0, 0, 2*z_PbTiO3])

# combine atoms together
atoms = PbTiO3 + SrTiO3

# fir lattice vectors now
cell[2][2] = 2*z_PbTiO3 + 3*z_SrTiO3
atoms.set_cell(cell)

ase.visualize.view(atoms)
ase.io.write('superlattice_POSCAR', atoms, format='vasp')

In [6]:
atoms = ase.io.read('PbTiO3_POSCAR')
atoms *= (4,4,4)
Pb_indices = [atom.index for atom in atoms if atom.symbol=="Pb"]
to_delete_index = Pb_indices[numpy.random.randint(0,len(Pb_indices))]
#del atoms[to_delete_index]
atoms[to_delete_index].symbol = "Sr"
ase.visualize.view(atoms)
ase.io.write('vacancy_POSCAR', atoms, format='vasp')

In [7]:
atoms = ase.io.read('PbTiO3_POSCAR')
atoms *= (4,4,4)

scaled_positions = atoms.get_scaled_positions()
positions = atoms.get_positions()
symbols = atoms.get_chemical_symbols()

# find Pb atom which is suffieicnetly away from the boundaries
Pb_indices = []
for i in range(len(symbols)):
    if symbols[i] != "Pb":
        continue
    if scaled_positions[i][0] > 0.75 or scaled_positions[i][0] < 0.25:
        continue
    if scaled_positions[i][1] > 0.75 or scaled_positions[i][1] < 0.25:
        continue
    if scaled_positions[i][2] > 0.75 or scaled_positions[i][2] < 0.25:
        continue
    Pb_indices.append(i)    

# randomly choose Pb atom from all potential Pb atoms   
desired_Pb_index = Pb_indices[numpy.random.randint(0,len(Pb_indices))]  

# find neighboring Ti and get its index
neighbor_dist = 1e+6   # initialize to some large number
neighbor_index = -1
for i in range(len(symbols)):
    if symbols[i] != 'Ti':
        continue
    dist = numpy.sum((numpy.array(positions[i]) - numpy.array(positions[desired_Pb_index]))**2)
    dist = numpy.sqrt(dist)
    if dist < neighbor_dist:
        neighbor_dist = dist
        neighbor_index = i

symbols[desired_Pb_index] = 'Ti'
symbols[neighbor_index] = 'Pb'
atoms.set_chemical_symbols(symbols)
ase.visualize.view(atoms)
ase.io.write('antisite_POSCAR', atoms, format='vasp')